In [1]:
# Install necessary libraries if not already installed
!pip install -q transformers datasets tqdm

# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import re
from tqdm import tqdm
import random

# Check if CUDA (GPU) is available, and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the GPT-Neo model and tokenizer, and move the model to the device
model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the MedQA dataset with training and testing splits
dataset = load_dataset("GBaker/MedQA-USMLE-4-options")
train_dataset = dataset['train']
test_dataset = dataset['test']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 14.2 MB/s eta 0:00:00
Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [2]:
# Function to create few-shot examples from the training data
def create_few_shot_examples(train_data, num_examples):
    """
    Create a few-shot prompt using num_examples samples from the training dataset.
    Each example will include a question, options, and the correct answer.
    """
    # Shuffle and select random examples from the training set for diversity
    shuffled_data = random.sample(list(train_data), len(train_data))

    examples = ""
    selected_categories = set()  # Keep track of categories to ensure diversity

    # Use tqdm to monitor the progress of few-shot example creation
    for sample in tqdm(shuffled_data, desc="Creating few-shot examples", total=len(train_data)):
        question = sample['question']
        options = sample['options']
        correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

        # Ensure diversity by limiting to different topics or question types (if available)
        if len(selected_categories) >= num_examples:
            break

        # Prepare the few-shot format (Question, Options, Answer)
        options_text = ', '.join([f"{key}: {value}" for key, value in options.items()])
        example = f"Question: {question}\nOptions: {options_text}\nAnswer: {correct_answer}\n\n"
        examples += example

        # Use some logic to track different question types/domains (optional)
        selected_categories.add(question)  # Replace with refined logic for topic detection

    return examples

# Function to perform inference with few-shot prompting and match answer using regex
def get_model_prediction(question, options_dict, few_shot_prompt):
    """
    Generate a few-shot prediction using GPT-Neo for a question with multiple choices.
    The options are provided as a dictionary with keys: 'A', 'B', 'C', 'D'.
    The few-shot examples are passed as part of the prompt.
    """
    # Prepare the prompt with few-shot examples + the current question
    options_text = ', '.join([f"{key}: {value}" for key, value in options_dict.items()])
    input_text = f"{few_shot_prompt}\nQuestion: {question}\nOptions: {options_text}\nAnswer:"''

    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output (can be tuned using temperature, max_length, etc.)
    output = model.generate(inputs["input_ids"], max_new_tokens=200)

    # Decode the output to text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)


    ind=generated_text.find(question)
    extracted_text=generated_text[ind+len(question)+len(options_text)+10:]
    ind2=extracted_text.find("Question")
    predicted=extracted_text[:ind2].strip()

    return predicted

In [3]:
total_predictions = 155  # Total number of predictions made up to iteration 86
accuracy_at_155 = 61.29  # Accuracy at iteration 86
correct_predictions = int((accuracy_at_155 / 100) * total_predictions)

for i, sample in enumerate(tqdm(test_dataset, desc="Evaluating on test dataset", total=len(test_dataset))):
    if i < 155:
        continue

    few_shot_prompt = create_few_shot_examples(train_dataset, num_examples=3)
    question = sample['question']
    options = sample['options']  # This is a dictionary with 'A', 'B', 'C', 'D' as keys
    correct_answer = sample['answer']  # Correct answer is the key ('A', 'B', 'C', or 'D')

    # Get the model's prediction using few-shot prompting
    predicted_answer = get_model_prediction(question, options, few_shot_prompt)

    print(predicted_answer)
    print(correct_answer)

    # Check if prediction is correct
    if correct_answer in predicted_answer:
        correct_predictions += 1

    total_predictions += 1

    # Calculate and print the accuracy after each iteration
    accuracy = correct_predictions / total_predictions * 100
    print(f"Iteration {i+1}: Accuracy so far: {accuracy:.2f}%")


Creating few-shot examples:   0%|          | 3/10178 [00:00<00:00, 17073.15it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.
Evaluating on test dataset:  12%|█▏        | 156/1273 [01:02<07:27,  2.50it/s]

Answer: Synthetic cathinone intoxication
Synthetic cathinone intoxication
Iteration 156: Accuracy so far: 60.90%



Evaluating on test dataset:  12%|█▏        | 157/1273 [01:56<16:32,  1.12it/s]

Answer: Benztropine
Benztropine
Iteration 157: Accuracy so far: 61.15%



Evaluating on test dataset:  12%|█▏        | 158/1273 [02:51<29:25,  1.58s/it]

...

Answer: Abnormal transfer of phosphate to cellular proteins
Impaired degradation of β-catenin
Iteration 312: Accuracy so far: 54.81%



Evaluating on test dataset:  25%|██▍       | 313/1273 [2:40:18<15:28:14, 58.01s/it]

Answer: Schistocytes
Linear aggregations of red blood cells
Iteration 313: Accuracy so far: 54.63%



Evaluating on test dataset:  25%|██▍       | 313/1273 [2:40:52<8:13:23, 30.84s/it] 


KeyboardInterrupt: 